In [1]:
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

db_path = 'chinook.db'
engine = create_engine('sqlite:///{}'.format(db_path))

In [2]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [3]:
albums = Base.classes.albums
artists = Base.classes.artists
customers = Base.classes.customers
invoice_items = Base.classes.invoice_items
invoices = Base.classes.invoices
tracks = Base.classes.tracks

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
result = session \
    .query(artists.Name, func.sum(invoice_items.Quantity * invoice_items.UnitPrice)) \
    .join(tracks, tracks.TrackId == invoice_items.TrackId) \
    .join(albums, albums.AlbumId == tracks.AlbumId) \
    .join(artists, artists.ArtistId == albums.ArtistId) \
    .join(invoices, invoices.InvoiceId == invoice_items.InvoiceId) \
    .join(customers, customers.CustomerId == invoices.CustomerId) \
    .filter_by(State='CA') \
    .group_by(artists.Name) \
    .order_by(func.sum(invoice_items.Quantity * invoice_items.UnitPrice).desc()) \
    .limit(3)

In [6]:
for i in result:
    print(i)

('Iron Maiden', Decimal('19.80'))
('Eric Clapton', Decimal('8.91'))
('The Office', Decimal('5.97'))


/opt/miniconda3/envs/fin-env/lib/python3.8/site-packages/sqlalchemy/sql/sqltypes.py:661: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(
